In [1]:
from collections import namedtuple, deque
import random
import torch
import math
from torch import nn
from torch import optim

In [2]:
def create_board():
    us = torch.zeros(3, 3)
    them = torch.zeros(3, 3)
    return torch.stack([us, them])

def get_move_coords(move_index):
    return move_index % 3, move_index // 3

def is_valid_move(state, move_index):
    if move_index < 0 or move_index >= 9:
        return False
    x, y = get_move_coords(move_index)
    return torch.all(state[:, y, x] == 0).item()

def make_move(state, move_index):
    assert(is_valid_move(state, move_index))
    x, y = get_move_coords(move_index)
    us, them = state.unbind()
    us = us.clone()
    us[y, x] = 1
    return torch.stack([them, us])

def has_won(ply):
    for x in range(3):
        if torch.sum(ply[:, x]) == 3:
            return True
    for y in range(3):
        if torch.sum(ply[y, :]) == 3:
            return True
    if sum([ply[i, i].item() for i in range(3)]) == 3:
        return True
    if sum([ply[2 - i, i].item() for i in range(3)]) == 3:
        return True
    return False

def get_score(state):
    us, them = state.unbind()
    if has_won(us):
        return 1 
    if has_won(them):
        return -1
    return 0

def is_complete(state):
    if torch.sum(state, (-3, -2, -1)) == 9:
        return True
    return get_score(state) != 0

In [3]:
state = create_board()

In [4]:
state

tensor([[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]])

In [5]:
is_valid_move(state, 6)

True

In [6]:
state = make_move(state, 6)

In [7]:
state

tensor([[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [1., 0., 0.]]])

In [8]:
get_score(state)

0

In [9]:
state = make_move(state, 0)

In [10]:
state = make_move(state, 3)

In [11]:
state

tensor([[[1., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [1., 0., 0.],
         [1., 0., 0.]]])

In [12]:
state = make_move(state, 1)

In [13]:
get_score(state)

0

In [14]:
state = make_move(state, 5)

In [15]:
get_score(state)

0

In [16]:
state

tensor([[[1., 1., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [1., 0., 1.],
         [1., 0., 0.]]])

In [17]:
is_complete(state)

False

In [18]:
state = make_move(state, 2)

In [19]:
state

tensor([[[0., 0., 0.],
         [1., 0., 1.],
         [1., 0., 0.]],

        [[1., 1., 1.],
         [0., 0., 0.],
         [0., 0., 0.]]])

In [20]:
is_complete(state)

True

In [21]:
get_score(state)

-1

# The Q equation

`Q(state, a) => reward`

`GAMMA` is a discount factor for future rewards.

If `state'` is non-final:

`Q(state, a) = reward + GAMMA * max_{a'} Q(state', a')`

Otherwise, `state'` is final, which means no more actions are possible:

`Q(state, a) = reward`

What move should I make?

In [25]:
# self.number_of_actions = 2
# self.gamma = 0.99
# self.final_epsilon = 0.0001
# self.initial_epsilon = 0.1
# self.number_of_iterations = 2000000
# self.replay_memory_size = 10000
# self.minibatch_size = 32

number_of_actions = 9

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(18, 32)
        self.relu1 = nn.ReLU(inplace=True)
        self.fc2 = nn.Linear(32, 32)
        self.relu2 = nn.ReLU(inplace=True)
        self.fc3 = nn.Linear(32, number_of_actions)

    def forward(self, x):
        out = x.view(x.size()[0], -1)
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        return out

In [26]:
model = NeuralNetwork()

In [27]:
minibatch = torch.stack([create_board(), state])

In [28]:
minibatch

tensor([[[[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],

         [[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]],


        [[[0., 0., 0.],
          [1., 0., 1.],
          [1., 0., 0.]],

         [[1., 1., 1.],
          [0., 0., 0.],
          [0., 0., 0.]]]])

In [29]:
model.forward(minibatch)

tensor([[-0.0858, -0.0897,  0.1664,  0.0675, -0.1441, -0.1525,  0.0551, -0.0116,
         -0.1083],
        [-0.0430, -0.1351,  0.2057,  0.0796, -0.1263, -0.1590, -0.0248, -0.0049,
         -0.1217]], grad_fn=<AddmmBackward0>)

In [30]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):
    def __init__(self, capacity):
        self.memory = deque([],maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)


In [31]:
optimizer = optim.RMSprop(model.parameters())
memory = ReplayMemory(10000)

In [32]:
BATCH_SIZE = 128
GAMMA = 0.999
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 200
TARGET_UPDATE = 10

In [85]:
steps_done = 0

def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max(1) will return largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            return torch.stack([model(torch.stack([state])).max(1)[1]])
    else:
        return torch.tensor([[random.randrange(number_of_actions)]], dtype=torch.long)


In [82]:
select_action(state)

tensor([[2]])

In [86]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None, batch.next_state)), dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to the model
    state_action_values = model(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1)[0].
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE)
    next_state_values[non_final_mask] = model(non_final_next_states).max(1)[0].detach()
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    for param in model.parameters():
        param.grad.data.clamp_(-1, 1)
    optimizer.step()


In [87]:
num_episodes = 50
for i_episode in range(num_episodes):
    # Initialize the environment and state
    state = create_board()
    while state is not None:
        # Select and perform an action
        action = select_action(state)
        raw_action = action.item()
        if not is_valid_move(state, raw_action):
            next_state = None
            reward = -1
            done = True
        else:
            next_state = make_move(state, raw_action)
            reward = -get_score(next_state)
            done = is_complete(next_state)
        reward = torch.tensor([reward])
        if done:
            next_state = None

        # Store the transition in memory
        print(action)
        memory.push(state, action, next_state, reward)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the policy network)
        optimize_model()


tensor([[5]])


TypeError: expected Tensor as element 2 in argument 0, but got int